In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from config import ex, cfg
from torchvision.transforms import Compose
from dataloaders.customized import voc_fewshot, coco_fewshot
from dataloaders.transforms import RandomMirror, Resize, ToTensorNormalize
from dataloaders.transforms import RandomMirror, Resize, ToTensorNormalize
from util.utils import set_seed, CLASS_LABELS

In [ ]:
_config = cfg()
data_name = _config['dataset']
transforms = Compose([Resize(size=_config['input_size']),
                        RandomMirror()])
labels = CLASS_LABELS[data_name][_config['label_sets']]

dataset = voc_fewshot(
    base_dir=_config['path'][data_name]['data_dir'],
    split=_config['path'][data_name]['data_split'],
    transforms=transforms,
    to_tensor=ToTensorNormalize(),
    labels=labels,
    max_iters=_config['n_steps'] * _config['batch_size'],
    n_ways=_config['task']['n_ways'],
    n_shots=_config['task']['n_shots'],
    n_queries=_config['task']['n_queries']
)

In [ ]:
data = dataset[0]

In [ ]:
data.keys()

In [ ]:

support_image = data['support_images']
support_mask = data['support_mask']
n_way = len(support_image)
n_shot = len(support_image[0])
print(n_way, n_shot)

img = np.moveaxis(support_image[0][0].numpy(), 0, -1)
plt.imshow(img)
plt.show()

mask = support_mask[0][0]['bg_mask'].numpy()
plt.imshow(mask)
plt.show()

## Test

In [ ]:
"""Evaluation Script"""
import os
import shutil

from config import ex, cfg
import tqdm
import numpy as np
import torch
import torch.optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
from torchvision.transforms import Compose

from models.fewshot import FewShotSeg
from dataloaders.customized import voc_fewshot, coco_fewshot
from dataloaders.transforms import ToTensorNormalize
from dataloaders.transforms import Resize, DilateScribble
from util.metric import Metric
from util.utils import set_seed, CLASS_LABELS, get_bbox
from config import ex

_config = cfg()
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

model = FewShotSeg(pretrained_path=_config['path']['init_path'], cfg=_config['model'])
model = nn.DataParallel(model.cuda())
if not _config['notrain']:
    snapshot = './runs/PANet_VOC_align_sets_0_1way_1shot_[train]/3/snapshots/30000.pth'
    print('Load PANet model from ', snapshot)
    model.load_state_dict(torch.load(snapshot, map_location='cpu'))
model.eval()

alp_model_config = _config['model']
alp_model_config['use_alp'] = True
model_alp = FewShotSeg(pretrained_path=_config['path']['init_path'], cfg=alp_model_config)
model_alp = nn.DataParallel(model_alp.cuda())
if not _config['notrain']:
    snapshot = './runs/PANet_VOC_align_use_alp_sets_0_1way_1shot_masked_no_back_alp_4_[train]/1/snapshots/30000.pth'
    print('Load alp model from ', snapshot)
    model_alp.load_state_dict(torch.load(snapshot, map_location='cpu'))
model_alp.eval()

data_name = _config['dataset']
if data_name == 'VOC':
    make_data = voc_fewshot
    max_label = 20
elif data_name == 'COCO':
    make_data = coco_fewshot
    max_label = 80
else:
    raise ValueError('Wrong config for dataset!')
labels = CLASS_LABELS[data_name]['all'] - CLASS_LABELS[data_name][_config['label_sets']]
transforms = [Resize(size=_config['input_size'])]
if _config['scribble_dilation'] > 0:
    transforms.append(DilateScribble(size=_config['scribble_dilation']))
transforms = Compose(transforms)


metric = Metric(max_label=max_label, n_runs=_config['n_runs'])
metric_alp = Metric(max_label=max_label, n_runs=_config['n_runs'])
with torch.no_grad():
    run = 0
    # set_seed(_config['seed'] + run)

    print(f'### Load data ###')
    dataset = make_data(
        base_dir=_config['path'][data_name]['data_dir'],
        split=_config['path'][data_name]['data_split'],
        transforms=transforms,
        to_tensor=ToTensorNormalize(),
        labels=labels,
        max_iters=_config['n_steps'] * _config['batch_size'],
        n_ways=_config['task']['n_ways'],
        n_shots=_config['task']['n_shots'],
        n_queries=_config['task']['n_queries']
    )
    if _config['dataset'] == 'COCO':
        coco_cls_ids = dataset.datasets[0].dataset.coco.getCatIds()
    testloader = DataLoader(dataset, batch_size=_config['batch_size'], shuffle=True,
                            num_workers=1, pin_memory=True, drop_last=False)
    print(f"Total # of Data: {len(dataset)}")

    iter_object = testloader.__iter__()
    sample_batched = next(iter_object)
    sample_batched = next(iter_object)

    if _config['dataset'] == 'COCO':
        label_ids = [coco_cls_ids.index(x) + 1 for x in sample_batched['class_ids']]
    else:
        label_ids = list(sample_batched['class_ids'])
    support_images = [[shot.cuda() for shot in way]
                        for way in sample_batched['support_images']]
    suffix = 'scribble' if _config['scribble'] else 'mask'

    if _config['bbox']:
        support_fg_mask = []
        support_bg_mask = []
        for i, way in enumerate(sample_batched['support_mask']):
            fg_masks = []
            bg_masks = []
            for j, shot in enumerate(way):
                fg_mask, bg_mask = get_bbox(shot['fg_mask'],
                                            sample_batched['support_inst'][i][j])
                fg_masks.append(fg_mask.float().cuda())
                bg_masks.append(bg_mask.float().cuda())
            support_fg_mask.append(fg_masks)
            support_bg_mask.append(bg_masks)
    else:
        support_fg_mask = [[shot[f'fg_{suffix}'].float().cuda() for shot in way]
                            for way in sample_batched['support_mask']]
        support_bg_mask = [[shot[f'bg_{suffix}'].float().cuda() for shot in way]
                            for way in sample_batched['support_mask']]

    query_images = [query_image.cuda()
                    for query_image in sample_batched['query_images']]
    query_labels = torch.cat(
        [query_label.cuda()for query_label in sample_batched['query_labels']], dim=0)

    query_pred_alp, _ = model_alp(support_images, support_fg_mask, support_bg_mask,
                            query_images)
                            
    query_pred, _ = model(support_images, support_fg_mask, support_bg_mask,
                            query_images)

    metric.record(np.array(query_pred.argmax(dim=1)[0].cpu()),
                    np.array(query_labels[0].cpu()),
                    labels=label_ids, n_run=run)

    classIoU, meanIoU = metric.get_mIoU(labels=sorted(labels), n_run=run)
    classIoU_binary, meanIoU_binary = metric.get_mIoU_binary(n_run=run)
    print('Original')
    print(f'classIoU: {classIoU}')
    print(f'meanIoU: {meanIoU}')
    print(f'classIoU_binary: {classIoU_binary}')
    print(f'meanIoU_binary: {meanIoU_binary}')

    print('Adaptive Local pooling')
    metric_alp.record(np.array(query_pred_alp.argmax(dim=1)[0].cpu()),
                    np.array(query_labels[0].cpu()),
                    labels=label_ids, n_run=run)

    classIoU, meanIoU = metric_alp.get_mIoU(labels=sorted(labels), n_run=run)
    classIoU_binary, meanIoU_binary = metric_alp.get_mIoU_binary(n_run=run)
    print(f'classIoU: {classIoU}')
    print(f'meanIoU: {meanIoU}')
    print(f'classIoU_binary: {classIoU_binary}')
    print(f'meanIoU_binary: {meanIoU_binary}')

In [ ]:
plt.figure()

plt.subplot(1, 2, 1)
plt.imshow(support_fg_mask[0][0][0].cpu().numpy())
plt.subplot(1, 2, 2)

support_image = support_images[0][0][0].permute(1, 2, 0).cpu().numpy()
support_image = (support_image - support_image.min()) / (support_image.max() - support_image.min())
plt.imshow(support_image)
plt.title('Support image')
plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(query_labels[0].cpu().numpy())
plt.subplot(1, 2, 2)

query_image = query_images[0][0].permute(1, 2, 0).cpu().numpy()
query_image = (query_image - query_image.min()) / (query_image.max() - query_image.min())
plt.imshow(query_image)
plt.title('Query image')
plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(query_pred.softmax(dim=1)[0][1].cpu().numpy())
plt.title(f'PANet (IoU = {round(np.nanmean(metric.get_mIoU(labels=sorted(labels), n_run=run)[0]), 3)})')
plt.colorbar(shrink=0.5)

plt.subplot(1, 2, 2)
plt.imshow(query_pred_alp.softmax(dim=1)[0][1].cpu().numpy())
plt.title(f'PANet + ALP (IoU = {round(np.nanmean(metric_alp.get_mIoU(labels=sorted(labels), n_run=run)[0]), 3)})')
plt.colorbar(shrink=0.5)
plt.show()
